In [1]:
%run ../load_files.ipynb

In [2]:
%run ../data_dti_matrixes.ipynb

(140, 39)


In [3]:
%run ../y_multilevel.ipynb

,attention,memory
ID,,
01_SLD,0,0
02_MSG,0,0
03_JAC,0,0
04_NSV,0,0
05_EGR,0,0


In [4]:
y = y_values.memory

In [5]:
# Training and test subsamples

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, stratify=y)

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(112, 39)
(28, 39)
(112,)
(28,)


In [7]:
# Keras model

from keras.models import Sequential
from keras.layers import Dense, Activation, InputLayer
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from scipy import random
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier

n_classes = 2


def create_model(n_hidden, learning_rate):
    model = Sequential()
    model.add(InputLayer(input_shape=(2850, )))
    model.add(Dense(units=n_hidden))
    model.add(Activation("sigmoid"))
    model.add(Dense(n_classes))
    model.add(Activation("softmax"))
    sgd = SGD(lr=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=["accuracy"])
    return model

params = {
    'n_hidden': random.randint(5, 200, 100),
    'learning_rate': random.uniform(0.001, 0.2, 100),
    'epochs': random.randint(10, 50, 100),
}
fn_model = KerasClassifier(build_fn=create_model)

from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

from neudataload.transformer import CombineMatrixTransformer, SpreadOutMatrixTransformer, FeatureMatrixTransformer
from neudataload.utils import all_combinations

# Pipeline configuration
pipe = Pipeline([
    ('combining', CombineMatrixTransformer(column_name='combined')),
    ('spread_out', SpreadOutMatrixTransformer(columns=['combined'])),
    ('filter', FeatureMatrixTransformer(matrix_columns=['combined'])),

    # ('reduce_dim', PCA(n_components=43)),
    ('classify', fn_model)
])


# Parametrized values
N_FEATURES_OPTIONS = range(1, 500, 10)
MATRIXES = list(all_combinations(['DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX', ]))
EXTRA_1D_COLUMS = [None]                      


param_grid = { 
        'combining__columns': MATRIXES,
        # 'filter__columns': EXTRA_1D_COLUMS,
        
        'classify__n_hidden': random.randint(20, 200, 1000),
        'classify__learning_rate': random.uniform(0.001, 0.2, 1000),
        'classify__epochs': random.randint(10, 50, 1000),
    }

Using TensorFlow backend.


In [8]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

# Search the best parameter combination with Cross Validation 
cv = StratifiedKFold(n_splits=3, shuffle=True)

grid = RandomizedSearchCV(pipe, cv=cv, n_jobs=-1, param_distributions=param_grid, 
                          verbose=3,
                          scoring='accuracy',
                          n_iter = 150)


In [9]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 150 candidates, totalling 450 fits
[CV] combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), classify__n_hidden=89, classify__learning_rate=0.14583267802375088, classify__epochs=13 
[CV] combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), classify__n_hidden=89, classify__learning_rate=0.14583267802375088, classify__epochs=13 
[CV] combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), classify__n_hidden=89, classify__learning_rate=0.14583267802375088, classify__epochs=13 
[CV] combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX'), classify__n_hidden=63, classify__learning_rate=0.09369589108338279, classify__epochs=27 
[CV] combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX'), classify__n_hidden=63, classify__learning_rate=0.09369589108338279, classify__epochs=27 
[CV] combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX'), classify__n_hidden=63, classify__learning_rate=0.09369589108338279, classify__epochs=27 
[CV] combining__columns=('DTI_L1', 'DTI_MD'), classify__n_hidden=62

32/75 [===========>..................] - ETA: 0s - loss: 0.5369 - acc: 0.7500Epoch 12/27
Epoch 2/20
75/75 [==============================] - 0s 272us/step - loss: 0.5862 - acc: 0.7467
Epoch 12/13
Epoch 10/27
75/75 [==============================] - 0s 158us/step - loss: 0.5670 - acc: 0.7467
Epoch 11/27
Epoch 3/20
74/74 [==============================] - 0s 485us/step - loss: 0.5611 - acc: 0.7568
Epoch 12/27
Epoch 4/20
Epoch 9/27
75/75 [==============================] - 0s 315us/step - loss: 0.5672 - acc: 0.7467
Epoch 15/27
Epoch 13/27
74/74 [==============================] - 0s 280us/step - loss: 0.5490 - acc: 0.7568
Epoch 16/27
75/75 [==============================] - 0s 4ms/step - loss: 0.7476 - acc: 0.5600
Epoch 11/27
Epoch 6/20
Epoch 17/27
74/74 [==============================] - 0s 218us/step - loss: 0.5269 - acc: 0.7568
Epoch 15/27
75/75 [==============================] - 0s 257us/step - loss: 0.5828 - acc: 0.7467
Epoch 7/20
Epoch 18/27
32/75 [===========>..................] - ET

74/74 [==============================] - 0s 320us/step - loss: 0.5149 - acc: 0.7568
Epoch 24/27
75/75 [==============================] - 0s 427us/step - loss: 0.5776 - acc: 0.7467
Epoch 27/27
75/75 [==============================] - 0s 430us/step - loss: 0.5463 - acc: 0.7467
Epoch 13/20
32/74 [===========>..................] - ETA: 0s - loss: 0.4531 - acc: 0.8125Epoch 17/20
Epoch 20/27
75/75 [==============================] - 0s 353us/step - loss: 0.5409 - acc: 0.7467
Epoch 18/20
75/75 [==============================] - 0s 409us/step - loss: 0.5405 - acc: 0.7467
Epoch 14/20
Epoch 25/27
75/75 [==============================] - 0s 415us/step - loss: 0.5788 - acc: 0.7467
Epoch 15/20
Epoch 19/20
Epoch 22/27
75/75 [==============================] - 0s 346us/step - loss: 0.5539 - acc: 0.7467
Epoch 20/20
Epoch 23/27
75/75 [==============================] - 0s 570us/step - loss: 0.5357 - acc: 0.7467
Epoch 24/27
75/75 [==============================] - 0s 510us/step - loss: 0.6063 - acc: 0.7467

Epoch 2/29
74/74 [==============================] - 0s 617us/step - loss: 0.5579 - acc: 0.7568
Epoch 12/20
Epoch 8/29
75/75 [==============================] - 0s 622us/step - loss: 0.5750 - acc: 0.7467
Epoch 13/20
Epoch 9/29
75/75 [==============================] - 0s 495us/step - loss: 0.5998 - acc: 0.7467
Epoch 4/29
75/75 [==============================] - 0s 435us/step - loss: 0.5677 - acc: 0.7467
Epoch 14/20
74/74 [==============================] - 0s 328us/step - loss: 0.5572 - acc: 0.7568
Epoch 2/29
75/75 [==============================] - 0s 370us/step - loss: 0.5675 - acc: 0.7467
Epoch 15/20
75/75 [==============================] - 0s 448us/step - loss: 0.5896 - acc: 0.7467
Epoch 3/29
74/74 [==============================] - 0s 361us/step - loss: 0.5566 - acc: 0.7568
Epoch 12/29
75/75 [==============================] - 0s 262us/step - loss: 0.6170 - acc: 0.7467
Epoch 7/29
75/75 [==============================] - 0s 251us/step - loss: 0.5663 - acc: 0.7467
Epoch 8/29
Epoch 17/20


75/75 [==============================] - 0s 698us/step - loss: 0.5637 - acc: 0.7467
Epoch 2/15
Epoch 4/15
75/75 [==============================] - 0s 662us/step - loss: 0.5681 - acc: 0.7467
Epoch 25/29
75/75 [==============================] - 0s 443us/step - loss: 0.5669 - acc: 0.7467
Epoch 15/29
75/75 [==============================] - 0s 567us/step - loss: 0.5742 - acc: 0.7467
Epoch 26/29
32/74 [===========>..................] - ETA: 0s - loss: 0.6707 - acc: 0.6562Epoch 3/15
Epoch 13/15
Epoch 5/15
75/75 [==============================] - 0s 342us/step - loss: 0.5751 - acc: 0.7467
Epoch 21/29
75/75 [==============================] - 0s 826us/step - loss: 0.5675 - acc: 0.7467
Epoch 4/15
75/75 [==============================] - 0s 399us/step - loss: 0.5629 - acc: 0.7467
Epoch 6/15
75/75 [==============================] - 0s 296us/step - loss: 0.6093 - acc: 0.7467
Epoch 22/29
32/75 [===========>..................] - ETA: 0s - loss: 0.5552 - acc: 0.7500Epoch 5/15
Epoch 28/29
75/75 [======

...

Epoch 2/41
Epoch 25/41
75/75 [==============================] - 0s 159us/step - loss: 0.5496 - acc: 0.7467
Epoch 16/41
75/75 [==============================] - 0s 204us/step - loss: 0.6259 - acc: 0.7467
Epoch 3/41
75/75 [==============================] - 0s 261us/step - loss: 0.5484 - acc: 0.7467
Epoch 17/41
75/75 [==============================] - 0s 231us/step - loss: 0.5641 - acc: 0.7467
Epoch 4/41
74/74 [==============================] - 0s 196us/step - loss: 0.5093 - acc: 0.7568
Epoch 18/41
Epoch 26/41
75/75 [==============================] - 0s 233us/step - loss: 0.5971 - acc: 0.7467
Epoch 19/41
75/75 [==============================] - 0s 138us/step - loss: 0.6350 - acc: 0.7467
Epoch 20/41
Epoch 5/41
74/74 [==============================] - 0s 431us/step - loss: 0.4968 - acc: 0.7568
Epoch 27/41
74/74 [==============================] - 0s 301us/step - loss: 0.5055 - acc: 0.7568
Epoch 28/41
75/75 [==============================] - 0s 288us/step - loss: 0.5600 - acc: 0.7467
Epoch 6/

75/75 [==============================] - 0s 160us/step - loss: 0.5137 - acc: 0.7733
Epoch 30/41
75/75 [==============================] - 0s 166us/step - loss: 0.5014 - acc: 0.7467
Epoch 31/41
75/75 [==============================] - 0s 146us/step - loss: 0.5763 - acc: 0.7200
Epoch 32/41
75/75 [==============================] - 0s 151us/step - loss: 0.4881 - acc: 0.7467
Epoch 33/41
75/75 [==============================] - 0s 141us/step - loss: 0.4924 - acc: 0.7467
Epoch 34/41
75/75 [==============================] - 0s 174us/step - loss: 0.4994 - acc: 0.7867
Epoch 35/41
75/75 [==============================] - 0s 140us/step - loss: 0.4842 - acc: 0.7467
Epoch 36/41
75/75 [==============================] - 0s 166us/step - loss: 0.5032 - acc: 0.7467
Epoch 37/41
75/75 [==============================] - 0s 196us/step - loss: 0.5543 - acc: 0.5867
Epoch 38/41
75/75 [==============================] - 0s 143us/step - loss: 0.4989 - acc: 0.7467
Epoch 39/41
75/75 [==============================] -

[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:  4.3min finished


Epoch 1/48
112/112 [==============================] - 0s 726us/step - loss: 0.5779 - acc: 0.7411
Epoch 2/48
112/112 [==============================] - 0s 115us/step - loss: 0.5577 - acc: 0.7500
Epoch 3/48
112/112 [==============================] - 0s 109us/step - loss: 0.5503 - acc: 0.7500
Epoch 4/48
112/112 [==============================] - 0s 109us/step - loss: 0.5460 - acc: 0.7500
Epoch 5/48
112/112 [==============================] - 0s 89us/step - loss: 0.5395 - acc: 0.7500
Epoch 6/48
112/112 [==============================] - 0s 122us/step - loss: 0.5446 - acc: 0.7500
Epoch 7/48
112/112 [==============================] - 0s 127us/step - loss: 0.5321 - acc: 0.7500
Epoch 8/48
112/112 [==============================] - 0s 99us/step - loss: 0.5406 - acc: 0.7500
Epoch 9/48
112/112 [==============================] - 0s 96us/step - loss: 0.5283 - acc: 0.7500
Epoch 10/48
112/112 [==============================] - 0s 106us/step - loss: 0.5194 - acc: 0.7500
Epoch 11/48
112/112 [===========

RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
          error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('combining', CombineMatrixTransformer(column_name='combined', columns=None,
             op=<function mean at 0x7f592019a840>)), ('spread_out', SpreadOutMatrixTransformer(columns=['combined'], symmetric=True)), ('filter', FeatureMatrixTransformer(columns=None, matrix_columns=['combined'])), ('classify', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f58f1dbb9e8>)]),
          fit_params=None, iid=True, n_iter=150, n_jobs=-1,
          param_distributions={'combining__columns': [('DTI_FA',), ('DTI_L1',), ('DTI_MD',), ('DTI_RX',), ('DTI_FA', 'DTI_L1'), ('DTI_FA', 'DTI_MD'), ('DTI_FA', 'DTI_RX'), ('DTI_L1', 'DTI_MD'), ('DTI_L1', 'DTI_RX'), ('DTI_MD', 'DTI_RX'), ('DTI_FA', 'DTI_L1', 'DTI_MD'), ('DTI_FA', 'DTI_L1', 'DTI_RX'), ('DTI_FA... array([0.03195, 0.07759, ..., 0.11813, 0.02775]), 'classify__epochs': array(

In [10]:
%run ../grid_results.ipynb

combining__columns = ('DTI_FA',)
classify__n_hidden = 88
classify__learning_rate = 0.03291384334119732
classify__epochs = 48
Epoch 1/48
112/112 [==============================] - 0s 726us/step - loss: 0.6762 - acc: 0.5714
Epoch 2/48
112/112 [==============================] - 0s 130us/step - loss: 0.5510 - acc: 0.7500
Epoch 3/48
112/112 [==============================] - 0s 120us/step - loss: 0.5581 - acc: 0.7500
Epoch 4/48
112/112 [==============================] - 0s 591us/step - loss: 0.5420 - acc: 0.7500
Epoch 5/48
112/112 [==============================] - 0s 110us/step - loss: 0.5349 - acc: 0.7500
Epoch 6/48
112/112 [==============================] - 0s 106us/step - loss: 0.5339 - acc: 0.7500
Epoch 7/48
112/112 [==============================] - 0s 126us/step - loss: 0.5374 - acc: 0.7500
Epoch 8/48
112/112 [==============================] - 0s 127us/step - loss: 0.5303 - acc: 0.7500
Epoch 9/48
112/112 [==============================] - 0s 139us/step - loss: 0.5227 - acc: 0.7500
Ep